In [1]:

import pandas as pd
import numpy as np
import os
import gc
import matplotlib.pyplot as plt
from tqdm import * 
import featuretools as ft
#----------------核心模型----------------
from catboost import CatBoostClassifier
from sklearn.linear_model import SGDRegressor, LinearRegression, Ridge
#----------------交叉验证----------------
from sklearn.model_selection import StratifiedKFold, KFold
#----------------评估指标----------------
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss
#----------------忽略报警----------------
import warnings
warnings.filterwarnings('ignore')


e:\anaconda\Lib\site-packages\dask\dataframe\_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 10.0.1 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [2]:
# 读取训练数据和测试数据
train_data_ads = pd.read_csv('./train/train_data_ads.csv')
train_data_feeds = pd.read_csv('./train/train_feeds.csv')

test_data_ads = pd.read_csv('./test/test_data_ads.csv')
test_data_feeds = pd.read_csv('./test/test_feeds.csv')

train_data_ads.head(10)


,log_id,label,user_id,age,sex,residence,city,city_rank,series_dev,series_group,...,ad_click_list_v001,ad_click_list_v002,ad_click_list_v003,ad_close_list_v001,ad_close_list_v002,ad_close_list_v003,pt_d,u_newsCatInterestsST,u_refreshTimes,u_feedLifeCycle
0,576480,0,1516217,5,2,32,179,5,34,7,...,14737^25802^32492^13240^33873,1126^1037^1730^1719^1382,305^327^344^317^301,24107,1218,173,20191111,16^173^168^151^65,0,16
1,576478,0,1516217,5,2,32,179,5,34,7,...,14737^25802^32492^13240^33873,1126^1037^1730^1719^1382,305^327^344^317^301,24107,1218,173,20191111,16^173^168^151^65,0,16
2,576475,0,1516217,5,2,32,179,5,34,7,...,14737^25802^32492^13240^33873,1126^1037^1730^1719^1382,305^327^344^317^301,24107,1218,173,20191111,16^173^168^151^65,0,16
3,576473,0,1516217,5,2,32,179,5,34,7,...,14737^25802^32492^13240^33873,1126^1037^1730^1719^1382,305^327^344^317^301,24107,1218,173,20191111,16^173^168^151^65,0,16
4,576406,0,1516217,5,2,32,179,5,34,7,...,14737^25802^32492^13240^33873,1126^1037^1730^1719^1382,305^327^344^317^301,24107,1218,173,20191111,16^173^168^151^65,0,16
5,576488,0,1516217,5,2,32,179,5,34,7,...,14737^25802^32492^13240^33873,1126^1037^1730^1719^1382,305^327^344^317^301,24107,1218,173,20191111,16^173^168^151^65,0,16
6,576490,0,1516217,5,2,32,179,5,34,7,...,14737^25802^32492^13240^33873,1126^1037^1730^1719^1382,305^327^344^317^301,24107,1218,173,20191111,16^173^168^151^65,0,16
7,576342,0,1516217,5,2,32,179,5,34,7,...,14737^25802^32492^13240^33873,1126^1037^1730^1719^1382,305^327^344^317^301,24107,1218,173,20191111,16^173^168^151^65,0,16
8,576349,0,1516217,5,2,32,179,5,34,7,...,14737^25802^32492^13240^33873,1126^1037^1730^1719^1382,305^327^344^317^301,24107,1218,173,20191111,16^173^168^151^65,0,16
9,576351,0,1516217,5,2,32,179,5,34,7,...,14737^25802^32492^13240^33873,1126^1037^1730^1719^1382,305^327^344^317^301,24107,1218,173,20191111,16^173^168^151^65,0,16


In [3]:
train_data_feeds.head(10)

,u_userId,u_phonePrice,u_browserLifeCycle,u_browserMode,u_feedLifeCycle,u_refreshTimes,u_newsCatInterests,u_newsCatDislike,u_newsCatInterestsST,u_click_ca2_news,...,e_ch,e_m,e_po,e_pl,e_rn,e_section,e_et,label,cillabel,pro
0,1519036,14,17,14,17,7,78^0^112^57^94,0,78^98^98^168^112,78^98^168^112^8,...,19,24,10,3047,1,0,20191116,1,-1,40
1,1519036,14,17,14,17,7,78^0^112^57^94,0,78^98^112^8^94,78^98^112^8^94,...,19,24,2,3047,4,0,20191116,-1,-1,0
2,1519036,14,17,14,17,7,78^0^112^57^94,0,78^98^112^8^94,78^98^112^8^94,...,19,24,5,3047,2,0,20191116,-1,-1,0
3,1519036,14,17,14,17,7,78^0^112^57^94,0,78^112^8^94^98,112^98^78^8^94,...,19,24,6,275,1,1,20191116,-1,-1,0
4,1519036,14,17,14,17,7,78^0^112^94^57,0,78^98^98^98^17,17^98^78^168^112,...,19,24,6,3047,1,1,20191116,1,-1,60
5,1519036,14,17,14,17,7,78^0^112^57^94,0,78^98^112^8^94,78^98^112^8^94,...,19,24,7,3047,4,0,20191116,-1,-1,0
6,1519036,14,17,14,17,7,78^0^112^57^94,0,78^112^8^94^98,112^98^78^8^94,...,19,24,11,275,1,0,20191116,-1,-1,0
7,1519036,14,17,14,17,7,78^0^112^57^94,0,78^98^112^8^94,78^98^112^8^94,...,19,24,5,3047,4,0,20191116,-1,-1,0
8,1519036,14,17,14,17,7,78^0^112^57^94,0,78^112^8^94^98,112^98^78^8^94,...,19,24,6,275,1,1,20191116,1,-1,100
9,1519036,14,17,14,17,7,78^0^112^57^94,0,78^112^8^94^98,112^98^78^8^94,...,19,24,7,275,1,1,20191116,-1,-1,0


In [4]:
# 合并数据
train_data_ads['istest'] = 0
test_data_ads['istest'] = 1
data_ads = pd.concat([train_data_ads, test_data_ads], axis=0, ignore_index=True)

train_data_feeds['istest'] = 0
test_data_feeds['istest'] = 1
data_feeds = pd.concat([train_data_feeds, test_data_feeds], axis=0, ignore_index=True)

del train_data_ads, test_data_ads, train_data_feeds, test_data_feeds
gc.collect()

20

In [5]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [6]:
# 自然数编码
def label_encode(series, series2):
    unique = list(series.unique())
    return series2.map(dict(zip(
        unique, range(series.nunique())
    )))
    
for col in ['ad_click_list_v001','ad_click_list_v002','ad_click_list_v003','ad_close_list_v001','ad_close_list_v002','ad_close_list_v003','u_newsCatInterestsST']:
    data_ads[col] = label_encode(data_ads[col], data_ads[col])

In [7]:
%%time
print('穿越特征 Starting...')
data_ads['month'] = data_ads['pt_d'].apply(lambda x: int(str(x)[4:6]))
data_ads['day'] = data_ads['pt_d'].apply(lambda x: int(str(x)[6:8]))
data_ads['date'] = data_ads['day']


def get_date_feature(data, gap_list=[1, 2, 3,4,5,6], col=['user_id']):

    for gap in gap_list:

        # 后面时间-当前时间
        data['ts_{}_{}_diff_next'.format('_'.join(col), gap)] = data.groupby(col)['date'].shift(-gap)
        data['ts_{}_{}_diff_next'.format('_'.join(col), gap)] = data['ts_{}_{}_diff_next'.format('_'.join(col), gap)] - \
                                                                data['date']

        # 前面时间-当前时间
        data['ts_{}_{}_diff_last'.format('_'.join(col), gap)] = data.groupby(col)['date'].shift(+gap)
        data['ts_{}_{}_diff_last'.format('_'.join(col), gap)] = data['date'] - data[
            'ts_{}_{}_diff_last'.format('_'.join(col), gap)]

        # 统计不为nan的值，做差前有曝光，做差后就不会为nan。
        data['ts_{}_{}_diff_next_count'.format('_'.join(col), gap)] = data.groupby(col)[
            'ts_{}_{}_diff_next'.format('_'.join(col), gap)].transform('count')
        data['ts_{}_{}_diff_last_count'.format('_'.join(col), gap)] = data.groupby(col)[
            'ts_{}_{}_diff_last'.format('_'.join(col), gap)].transform('count')

        # 统计时间差的平均值
        data['ts_{}_{}_diff_next_mean'.format('_'.join(col), gap)] = data.groupby(col)[
            'ts_{}_{}_diff_next'.format('_'.join(col), gap)].transform('mean')
        data['ts_{}_{}_diff_last_mean'.format('_'.join(col), gap)] = data.groupby(col)[
            'ts_{}_{}_diff_last'.format('_'.join(col), gap)].transform('mean')

        # 统计时间差的最大值
        data['ts_{}_{}_diff_next_max'.format('_'.join(col), gap)] = data.groupby(col)[
            'ts_{}_{}_diff_next'.format('_'.join(col), gap)].transform('max')
        data['ts_{}_{}_diff_last_max'.format('_'.join(col), gap)] = data.groupby(col)[
            'ts_{}_{}_diff_last'.format('_'.join(col), gap)].transform('max')

        # 统计时间差的最小值
        data['ts_{}_{}_diff_next_min'.format('_'.join(col), gap)] = data.groupby(col)[
            'ts_{}_{}_diff_next'.format('_'.join(col), gap)].transform('min')
        data['ts_{}_{}_diff_last_min'.format('_'.join(col), gap)] = data.groupby(col)[
            'ts_{}_{}_diff_last'.format('_'.join(col), gap)].transform('min')

    return data


def get_diff_date(data, gap_list=[1, 2, 3,4,5,6], col=['user_id'], con_list=[1], f='next'):
    for gap in gap_list:
        for con in con_list:
            data['ts_s_{}_{}_{}_next_{}'.format(f, '_'.join(col), gap, con)] = data.groupby(col)[
                'ts_{}_{}_diff_{}'.format('_'.join(col), con, f)].shift(-gap)
            data['ts_s_{}_{}_{}_next_{}'.format(f, '_'.join(col), gap, con)] = data['ts_s_{}_{}_{}_next_{}'.format(f,
                                                                                                                   '_'.join(
                                                                                                                       col),
                                                                                                                   gap,
                                                                                                                   con)] - \
                                                                               data['ts_{}_{}_diff_{}'.format(
                                                                                   '_'.join(col), con, f)]

            data['ts_s_{}_{}_{}_last_{}'.format(f, '_'.join(col), gap, con)] = data.groupby(col)[
                'ts_{}_{}_diff_{}'.format('_'.join(col), con, f)].shift(+gap)
            data['ts_s_{}_{}_{}_last_{}'.format(f, '_'.join(col), gap, con)] = data['ts_{}_{}_diff_{}'.format(
                '_'.join(col), con, f)] - data['ts_s_{}_{}_{}_last_{}'.format(f, '_'.join(col), gap, con)]

    return data


for col in [
    ['user_id'], ['task_id'], ['adv_id'],
    ['user_id', 'adv_id'], ['user_id', 'task_id'], ['user_id', 'creat_type_cd'],
    ['user_id', 'adv_prim_id'], ['user_id', 'inter_type_cd'], ['user_id', 'slot_id'],
    ['user_id', 'site_id'], ['user_id', 'spread_app_id']
]:
    print('_'.join(col), 'make', 'feature')
    data_ads = get_date_feature(data_ads, gap_list=[1, 2, 3,4,5,6], col=col)
    data_ads = get_diff_date(data_ads, gap_list=[1, 2, 3,4,5,6], col=col, con_list=[1], f='next')
    data_ads = get_diff_date(data_ads, gap_list=[1, 2, 3,4,5,6], col=col, con_list=[1], f='last')


print('穿越特征 Ending...')


穿越特征 Starting...
user_id make feature
task_id make feature
adv_id make feature
user_id_adv_id make feature
user_id_task_id make feature
user_id_creat_type_cd make feature
user_id_adv_prim_id make feature
user_id_inter_type_cd make feature
user_id_slot_id make feature
user_id_site_id make feature
user_id_spread_app_id make feature
穿越特征 Ending...
CPU times: total: 1min 36s
Wall time: 1min 36s


In [8]:
# 压缩使用内存
data_ads = reduce_mem_usage(data_ads)
print(f'data_ads.shape = {data_ads.shape}')
# Mem. usage decreased to 2351.47 Mb (69.3% reduction)

Mem. usage decreased to 5488.88 Mb (74.5% reduction)
data_ads.shape = (2930502, 963)


In [9]:

print(f'data_feeds.shape = ',data_feeds.shape)
print(f'data_ads.shape = ',data_ads.shape)
print('信息域特征：')
print(data_feeds.columns)

## 去重计数特征
cols = [f for f in data_feeds.columns if f not in [ 'u_phonePrice', 'u_browserLifeCycle', 'u_browserMode',
       'u_feedLifeCycle', 'u_refreshTimes', 'u_newsCatInterests','u_newsCatDislike','i_dislikeTimes','u_userId', 
       'i_dtype', 'e_ch', 'e_m', 'e_pl', 'e_rn', 'e_section', 'label', 'cillabel', 'pro', 'istest']]
print(f'用来构建nunique特征的源域变量有{len(cols)}个，分别是：{cols}')
for col in tqdm(cols):
    tmp = data_feeds.groupby(['u_userId'])[col].nunique().reset_index()  #无重复值
    tmp.columns = ['user_id', col+'_feeds_nuni']
    data_ads = data_ads.merge(tmp, on='user_id', how='left')

## 均值特征
cols = [f for f in data_feeds.columns if
        f not in ['istest', 'u_userId', 'u_newsCatInterests', 'u_newsCatDislike', 'u_newsCatInterestsST',
                  'u_click_ca2_news', 'i_docId', 'i_s_sourceId', 'i_entities']]
print(f'用来构建mean特征的源域变量有{len(cols)}个，分别是：{cols}')
for col in tqdm(cols):
    tmp = data_feeds.groupby(['u_userId'])[col].mean().reset_index()
    tmp.columns = ['user_id', col+'_feeds_mean']
    data_ads = data_ads.merge(tmp, on='user_id', how='left')
    
## 最小值
cols = [f for f in data_feeds.columns if
        f not in ['istest', 'u_userId', 'u_newsCatInterests', 'u_newsCatDislike', 'u_newsCatInterestsST',
                  'u_click_ca2_news', 'i_docId', 'i_s_sourceId', 'i_entities']]
print(f'用来构建min特征的源域变量有{len(cols)}个，分别是：{cols}')
for col in tqdm(cols):
    tmp = data_feeds.groupby(['u_userId'])[col].min().reset_index()
    tmp.columns = ['user_id', col+'_feeds_mean']
    data_ads = data_ads.merge(tmp, on='user_id', how='left')
    
## 最大值
cols = [f for f in data_feeds.columns if
        f not in ['istest', 'u_userId', 'u_newsCatInterests', 'u_newsCatDislike', 'u_newsCatInterestsST',
                  'u_click_ca2_news', 'i_docId', 'i_s_sourceId', 'i_entities']]
print(f'用来构建max特征的源域变量有{len(cols)}个，分别是：{cols}')
for col in tqdm(cols):
    tmp = data_feeds.groupby(['u_userId'])[col].max().reset_index()
    tmp.columns = ['user_id', col+'_feeds_mean']
    data_ads = data_ads.merge(tmp, on='user_id', how='left')
    

print(f'data_feeds.shape = ',data_feeds.shape)
print(f'data_ads.shape = ',data_ads.shape)

data_feeds.shape =  (970782, 29)
data_ads.shape =  (2930502, 963)
信息域特征：
Index(['u_userId', 'u_phonePrice', 'u_browserLifeCycle', 'u_browserMode',
       'u_feedLifeCycle', 'u_refreshTimes', 'u_newsCatInterests',
       'u_newsCatDislike', 'u_newsCatInterestsST', 'u_click_ca2_news',
       'i_docId', 'i_s_sourceId', 'i_regionEntity', 'i_cat', 'i_entities',
       'i_dislikeTimes', 'i_upTimes', 'i_dtype', 'e_ch', 'e_m', 'e_po', 'e_pl',
       'e_rn', 'e_section', 'e_et', 'label', 'cillabel', 'pro', 'istest'],
      dtype='object')
用来构建nunique特征的源域变量有10个，分别是：['u_newsCatInterestsST', 'u_click_ca2_news', 'i_docId', 'i_s_sourceId', 'i_regionEntity', 'i_cat', 'i_entities', 'i_upTimes', 'e_po', 'e_et']


  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:12<00:00,  1.26s/it]


用来构建mean特征的源域变量有20个，分别是：['u_phonePrice', 'u_browserLifeCycle', 'u_browserMode', 'u_feedLifeCycle', 'u_refreshTimes', 'i_regionEntity', 'i_cat', 'i_dislikeTimes', 'i_upTimes', 'i_dtype', 'e_ch', 'e_m', 'e_po', 'e_pl', 'e_rn', 'e_section', 'e_et', 'label', 'cillabel', 'pro']


100%|██████████| 20/20 [00:21<00:00,  1.08s/it]


用来构建min特征的源域变量有20个，分别是：['u_phonePrice', 'u_browserLifeCycle', 'u_browserMode', 'u_feedLifeCycle', 'u_refreshTimes', 'i_regionEntity', 'i_cat', 'i_dislikeTimes', 'i_upTimes', 'i_dtype', 'e_ch', 'e_m', 'e_po', 'e_pl', 'e_rn', 'e_section', 'e_et', 'label', 'cillabel', 'pro']


100%|██████████| 20/20 [00:24<00:00,  1.22s/it]


用来构建max特征的源域变量有20个，分别是：['u_phonePrice', 'u_browserLifeCycle', 'u_browserMode', 'u_feedLifeCycle', 'u_refreshTimes', 'i_regionEntity', 'i_cat', 'i_dislikeTimes', 'i_upTimes', 'i_dtype', 'e_ch', 'e_m', 'e_po', 'e_pl', 'e_rn', 'e_section', 'e_et', 'label', 'cillabel', 'pro']


100%|██████████| 20/20 [00:28<00:00,  1.43s/it]

data_feeds.shape =  (970782, 29)
data_ads.shape =  (2930502, 1033)


In [10]:
# 压缩使用内存
print(f'data_ads.shape = {data_ads.shape}')
print(f'data_feeds.shape = {data_feeds.shape}')
data_ads = reduce_mem_usage(data_ads)
# Mem. usage decreased to 2351.47 Mb (69.3% reduction)

data_ads.shape = (2930502, 1033)
data_feeds.shape = (970782, 29)
Mem. usage decreased to 5796.30 Mb (17.8% reduction)


In [11]:
# 划分训练集和测试集
cols = [f for f in data_ads.columns if f not in ['label','istest']]
X_train = data_ads[data_ads.istest==0][cols]
X_test = data_ads[data_ads.istest==1][cols]
Y_train = data_ads[data_ads.istest==0]['label']

print('X_train.shape = ', X_train.shape)
print('Y_train.shape = ', Y_train.shape)
print('X_test.shape = ', X_test.shape)

del data_ads, data_feeds
gc.collect()

val_counts = Y_train.value_counts()
print(val_counts)
ratio = val_counts[1] / len(Y_train)
print(f'正样本个数：{val_counts[1]}，样本总数：{Y_train.shape[0]}，点击率：{ratio*100}%')

X_train.shape =  (2526320, 1031)
Y_train.shape =  (2526320,)
X_test.shape =  (404182, 1031)
label
0.0    2504966
1.0      21354
Name: count, dtype: int64
正样本个数：21354，样本总数：2526320，点击率：0.8452610912315146%


In [12]:
def cv_model(clf, train_x, train_y, test_x, clf_name, seed=2022):
    
    kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

    train = np.zeros(train_x.shape[0])
    test = np.zeros(test_x.shape[0])

    cv_scores = []

    for i, (train_index, valid_index) in enumerate(kf.split(train_x, train_y)):
        print('************************************ Fold: {}************************************'.format(str(i+1)))
        trn_x, trn_y, val_x, val_y = train_x.iloc[train_index], train_y[train_index], train_x.iloc[valid_index], train_y[valid_index]
               
        params = {'learning_rate': 0.3, 'depth': 7, 'l2_leaf_reg': 10, 'bootstrap_type':'Bernoulli','random_seed':seed, 
                  'od_type': 'Iter', 'early_stopping_rounds' : 200, 'random_seed': 11, 'allow_writing_files': False, 'task_type':'GPU'}

        model = clf(iterations=20000, **params, eval_metric='AUC')
        model.fit(trn_x, trn_y, eval_set=(val_x, val_y),
                  metric_period=200,
                  cat_features=[], 
                  use_best_model=True, 
                  verbose=1)

        val_pred  = model.predict_proba(val_x)[:,1]
        test_pred = model.predict_proba(test_x)[:,1]
            
        train[valid_index] = val_pred
        test += test_pred / kf.n_splits
        cv_scores.append(roc_auc_score(val_y, val_pred))
        
        print('Fold: ', i+1, ', cv_scores: ', cv_scores)
       
    print("%s_score_list:" % clf_name, cv_scores)
    print("%s_score_mean:" % clf_name, np.mean(cv_scores))
    print("%s_score_std:" % clf_name, np.std(cv_scores))
    return train, test, model

In [13]:
%%time
cat_train, cat_test, model_cat = cv_model(CatBoostClassifier, X_train, Y_train, X_test, "CatBoost")

************************************ Fold: 1************************************
0:	test: 0.6095259	best: 0.6095259 (0)	total: 34ms	remaining: 11m 20s
200:	test: 0.7772835	best: 0.7773647 (190)	total: 4.42s	remaining: 7m 15s
400:	test: 0.7852572	best: 0.7852872 (398)	total: 8.58s	remaining: 6m 59s
600:	test: 0.7868898	best: 0.7873581 (502)	total: 12.7s	remaining: 6m 49s
800:	test: 0.7874788	best: 0.7879484 (752)	total: 16.8s	remaining: 6m 43s
1000:	test: 0.7866482	best: 0.7883538 (822)	total: 20.9s	remaining: 6m 37s
bestTest = 0.7883538306
bestIteration = 822
Shrink model to first 823 iterations.
Fold:  1 , cv_scores:  [0.7883538158459484]
************************************ Fold: 2************************************
0:	test: 0.6109670	best: 0.6109670 (0)	total: 71.5ms	remaining: 23m 49s
200:	test: 0.7790715	best: 0.7793621 (174)	total: 4.63s	remaining: 7m 36s
400:	test: 0.7798538	best: 0.7806683 (373)	total: 8.83s	remaining: 7m 11s
600:	test: 0.7813858	best: 0.7814359 (592)	total: 1

In [14]:
X_test['pctr'] = cat_test
X_test[['log_id','pctr']].to_csv('submission.csv', index=False)
print('Done.')

Done.
